<a href="https://colab.research.google.com/github/inspire-lab/CyberAI-labs/blob/main/category-PrivateAI/Differential-privacy-DP-SGD/DP_SGD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Differentially Private SGD Training on MNIST

This notebook demonstrates how to train a Convolutional Neural Network (CNN) on the MNIST dataset using Differentially Private Stochastic Gradient Descent (DP-SGD) with [TensorFlow Privacy (TF Privacy)](https://www.tensorflow.org/responsible_ai/privacy/guide) which is an open source library developed by teams in Google Research. The library includes implementations of commonly used TensorFlow Optimizers for training ML models with DP..

## Objective
To understand how differential privacy can be applied in training machine learning models, ensuring the model's robustness and data privacy.

---


## Step 1: Install Tensorflow and Tensorflow Privacy

In [ ]:
!pip install tensorflow==2.14.0
!pip install tensorflow-privacy==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
      Successfully uninstalled wrapt-1.16.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: google-auth-oauthlib
    Found existing ins

## Step 2: Import Necessary Libraries

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
import dp_accounting
import logging



## Step 2: Define Hyperparameters

We set key hyperparameters for training the model with DP-SGD. These include:
- `learning_rate`: The learning rate for the optimizer.
- `noise_multiplier`: Determines the noise added for differential privacy.
- `l2_norm_clip`: Clips the gradient to ensure that individual training points do not contribute too much.
- `batch_size`, `epochs`, and `microbatches`: Define the training batch size, epochs, and microbatches (must evenly divide batch_size).
    

In [ ]:

# Define hyperparameters
dpsgd = True  # Use DP-SGD if True, otherwise vanilla SGD
learning_rate = 0.15
noise_multiplier = 0.1
l2_norm_clip = 1.0
batch_size = 250
epochs = 2 # change this to check results with multiple epochs
microbatches = 250  # Must evenly divide batch_size



## Step 3: Define the Function to Compute Epsilon

The function `compute_epsilon` calculates the privacy budget (epsilon) based on the DP-SGD hyperparameters.
A lower epsilon indicates stronger privacy but may impact model accuracy.
    

In [ ]:

def compute_epsilon(steps):
    """Computes epsilon value for given hyperparameters."""
    if noise_multiplier == 0.0:
        return float('inf')
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
    accountant = dp_accounting.rdp.RdpAccountant(orders)

    sampling_probability = batch_size / 60000
    event = dp_accounting.SelfComposedDpEvent(
        dp_accounting.PoissonSampledDpEvent(
            sampling_probability,
            dp_accounting.GaussianDpEvent(noise_multiplier)), steps)

    accountant.compose(event)

    # Delta is set to 1e-5 because MNIST has 60000 training points.
    return accountant.get_epsilon(target_delta=1e-5)



## Step 4: Load and Preprocess MNIST Data

We load the MNIST dataset and normalize it to have values between 0 and 1. The data is also reshaped to fit the CNN input requirements.
    

In [ ]:

def load_mnist():
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
    test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    return train_data, train_labels, test_data, test_labels

train_data, train_labels, test_data, test_labels = load_mnist()


11490434/11490434 [==============================] - 0s 0us/step



## Step 5: Define and Compile the CNN Model

Here, we define a CNN model suitable for MNIST digit classification. We compile it with either DP-SGD or vanilla SGD based on the hyperparameters set.
    

In [ ]:

# Define a sequential Keras model
model = tf.keras.Sequential([
    layers.Conv2D(16, 8, strides=2, padding='same', activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPool2D(2, 1),
    layers.Conv2D(32, 4, strides=2, padding='valid', activation='relu'),
    layers.MaxPool2D(2, 1),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(10)
])

if dpsgd:
    optimizer =
#####################
#Your code goes here
#####################
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
else:
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# Compile model with Keras
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])



## Step 6: Train the Model

Using the configured optimizer and model, we train the model on the MNIST dataset.
    

In [ ]:

history =
#####################
#Your code goes here
#####################

Epoch 1/2
240/240 [==============================] - 2587s 11s/step - loss: 1.5328 - accuracy: 0.5811 - val_loss: 0.7562 - val_accuracy: 0.7605
Epoch 2/2
240/240 [==============================] - 2488s 10s/step - loss: 0.6057 - accuracy: 0.8102 - val_loss: 0.4870 - val_accuracy: 0.8543



## Step 7: Calculate and Print Privacy Budget

After training, we calculate the privacy budget (`epsilon`) to understand the privacy level achieved.
    

In [ ]:

if dpsgd:
    steps = epochs * 60000 // batch_size
    epsilon = compute_epsilon(steps)
    print(f"For delta=1e-5, the current epsilon is: {epsilon:.2f}")
else:
    print("Trained with vanilla non-private SGD optimizer")


For delta=1e-5, the current epsilon is: 2321.94



## Conclusion

In this notebook, we successfully trained a CNN on the MNIST dataset using Differentially Private SGD, a privacy-preserving technique.
We analyzed the privacy-accuracy trade-off by adjusting the noise and privacy parameters.
    